# QSFP/CMAC Loopback Test
---

## Aim/s
This notebook provides an example demonstrating how to use with Xilinx 100 GbE subsystem. The 100 GbE subsystem consists of a hardened IP, which is known as the CMAC. This hardened IP is available in Xilinx UltraScale+ devices.

## References
* [CMAC Documentation](https://www.xilinx.com/products/intellectual-property/cmac_usplus.html)
* [PYNQ docs](https://pynq.readthedocs.io/en/latest/index.html)

## Last revised
* 25Feb22 - Initial revision
---

### Hardware Setup

The RFSoC 4x2 contains one QSFP+ port in a horizontal cage. This port is capable of 100Gb/s using 4 lanes, which can each achieve 25Gb/s. To evaluate the QSFP/CMAC features of the RFSoC 4x2, a QSFP loopback module should be connected to the QSFP port. A suggested loopback module is the Amphenol QSFP passive loopback modules. Alternatively, this notebook can use internal CMAC loopback, which will allow you to evaluate the CMAC core without using external connections.

### Base overlay
Load the overlay first.

In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

### CMAC Configuration
If you don't have loopback modules, you can still experiment with the CMAC by enabling "Near-End PMA Loopback" in the cell below. This functionality routes the transmit path to the receive path internally.

* 0 = external loopback
* 1 = internal loopback 

In [2]:
base.cmac.internal_loopback = 1

Start the CMAC core.

In [3]:
base.cmac.start()

### Test Setup
An AXI DMA is used to transfer data between the PS and CMAC (via the PL). The CMAC core supports packet sizes from 64 to 9,000 bytes. Create a byte array for testing and store it in a numpy array.

In [4]:
import numpy as np

raw_str = ' /$$$$$$$  /$$     /$$ /$$   /$$  /$$$$$$ \n| $$__  $$|  $$   /$$/| $$$ | $$ /$$__  $$\n| $$  \\ $$ \\  $$ /$$/ | $$$$| $$| $$  \\ $$\n| $$$$$$$/  \\  $$$$/  | $$ $$ $$| $$  | $$\n| $$____/    \\  $$/   | $$  $$$$| $$  | $$\n| $$          | $$    | $$\\  $$$| $$/$$ $$\n| $$          | $$    | $$ \\  $$|  $$$$$$/\n|__/          |__/    |__/  \\__/ \\____ $$$\n                                      \\__/\n\n                                          \n                                         '
packets_in = np.array(list(raw_str.encode()))

Allocate input and output buffers with size equal to the numpy array size (in this case 472 bytes).

In [5]:
from pynq import allocate

dma_buf_in = allocate(packets_in.shape[0], dtype=np.uint8)
dma_buf_out = allocate(packets_in.shape[0], dtype=np.uint8)
dma_buf_in[:] = packets_in

### Perform CMAC Loopback Test
Perform the CMAC loopback test by running the AXI DMA.

In [6]:
base.dma.sendchannel.transfer(dma_buf_in)
base.dma.recvchannel.transfer(dma_buf_out)
base.dma.sendchannel.wait()
base.dma.recvchannel.wait()

### Test Verification
Retrieve the output buffer and decode to reveal the original message.

In [7]:
packets_out = bytes(dma_buf_out)
print(packets_out.decode())

    /   $   $   $   $   $   $   $           /   $   $                       /   $   $       /   $   $               /   $   $           /   $   $   $   $   $   $       
   |       $   $   _   _           $   $   |           $   $               /   $   $   /   |       $   $   $       |       $   $       /   $   $   _   _           $   $   
   |       $   $           \       $   $       \           $   $       /   $   $   /       |       $   $   $   $   |       $   $   |       $   $           \       $   $   
   |       $   $   $   $   $   $   $   /           \           $   $   $   $   /           |       $   $       $   $       $   $   |       $   $           |       $   $   
   |       $   $   _   _   _   _   /                   \           $   $   /               |       $   $           $   $   $   $   |       $   $           |       $   $   
   |       $   $                                           |       $   $                   |       $   $   \           $   $   $   |       $   

Alternatively, verify that the input and output buffers are equal.

In [8]:
np.array_equal(dma_buf_in,dma_buf_out)

True

Lastly, the CMAC has internal registers that report core statistics. Note that the registers automatically self-clear after reporting. Run `cmac.getStats(False)` instead to prevent them from clearing.

In [9]:
base.cmac.getStats()

{'tx': {'packets': 1,
  'good_packets': 1,
  'bytes': 1892,
  'good_bytes': 1892,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 1,
  'good_packets': 1,
  'bytes': 1892,
  'good_bytes': 1892,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 0,
  'packets_bad_fcs': 0,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 340659205}

---

Copyright (C) 2022 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

---
---